<a href="https://colab.research.google.com/github/nicholashobbs/Random-Notes/blob/master/Fantasy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd, numpy as np
print("pandas:", pd.__version__, "numpy:", np.__version__)
# Expect: pandas 2.2.2, numpy 1.26.4


pandas: 2.2.2 numpy: 2.0.2


In [4]:
# Cell 1 — install
%pip install --no-deps "nflreadpy" "polars" "pyarrow"


  Using cached nflreadpy-0.1.2-py3-none-any.whl.metadata (6.2 kB)


In [5]:
import polars as pl, nflreadpy as nfl
from importlib.metadata import version
print("nflreadpy:", version("nflreadpy"), "| polars:", pl.__version__)

SEASON = nfl.get_current_season()
WEEK   = nfl.get_current_week()
SEASON, WEEK


nflreadpy: 0.1.2 | polars: 1.25.2


(2025, 3)

In [9]:
sched = nfl.load_schedules(seasons=SEASON)

this_week = (
    sched
    .filter(pl.col("week") == WEEK)
    .select("game_id","season","week","gameday","gametime",
            "home_team","away_team","spread_line","total_line")
    .sort("gameday")
)
this_week.sample(5)


game_id,season,week,gameday,gametime,home_team,away_team,spread_line,total_line
str,i64,i64,str,str,str,str,f64,f64
"""2025_03_ATL_CAR""",2025,3,"""2025-09-21""","""13:00""","""CAR""","""ATL""",-5.5,43.5
"""2025_03_CIN_MIN""",2025,3,"""2025-09-21""","""13:00""","""MIN""","""CIN""",3.0,41.5
"""2025_03_DAL_CHI""",2025,3,"""2025-09-21""","""16:25""","""CHI""","""DAL""",-1.5,50.5
"""2025_03_IND_TEN""",2025,3,"""2025-09-21""","""13:00""","""TEN""","""IND""",-4.5,43.5
"""2025_03_ARI_SF""",2025,3,"""2025-09-21""","""16:25""","""SF""","""ARI""",2.5,45.5


In [10]:
wk = (
    this_week
    .with_columns([
        ((pl.col("total_line")/2) - (pl.col("spread_line")/2)).alias("home_it"),
        ((pl.col("total_line")/2) + (pl.col("spread_line")/2)).alias("away_it")
    ])
)

home = wk.select(["game_id","home_team","home_it"]).rename({"home_team":"team_clean","home_it":"implied_total"})
away = wk.select(["game_id","away_team","away_it"]).rename({"away_team":"team_clean","away_it":"implied_total"})
implied = pl.concat([home, away])
implied.head()


game_id,team_clean,implied_total
str,str,f64
"""2025_03_MIA_BUF""","""BUF""",18.5
"""2025_03_ATL_CAR""","""CAR""",24.5
"""2025_03_GB_CLE""","""CLE""",25.0
"""2025_03_HOU_JAX""","""JAX""",21.5
"""2025_03_CIN_MIN""","""MIN""",19.25


In [15]:
# Cell — load season-to-date player weekly + snaps, restricted to completed weeks
import polars as pl, nflreadpy as nfl

# Guard: make sure LAST_DONE_WEEK exists
try:
    LAST_DONE_WEEK
except NameError:
    LAST_DONE_WEEK = max(int(WEEK) - 1, 1)

player_weekly = nfl.load_player_stats(seasons=SEASON, summary_level="week")
snaps         = nfl.load_snap_counts(seasons=SEASON)

# Pick the correct team column that actually exists
team_col = "team" if "team" in player_weekly.columns else ("recent_team" if "recent_team" in player_weekly.columns else None)
if team_col is None:
    raise ValueError("No team column found in player_weekly. Available columns: " + ", ".join(player_weekly.columns))

# Prepare a minimal season-to-date table (no risky columns yet)
pw_hist = (
    player_weekly
    .filter(pl.col("week") <= LAST_DONE_WEEK)
    .with_columns([
        pl.col("player_display_name").alias("player_name"),
        pl.col(team_col).alias("team_clean")
    ])
)

# Show exactly what we have to work with (so we only reference real columns next)
print("player_weekly columns:\n", player_weekly.columns)
print("\nsnaps columns:\n", snaps.columns)

pw_hist.select(["player_name","position","team_clean","week"]).head(10)


player_weekly columns:
 ['player_id', 'player_name', 'player_display_name', 'position', 'position_group', 'headshot_url', 'season', 'week', 'season_type', 'team', 'opponent_team', 'completions', 'attempts', 'passing_yards', 'passing_tds', 'passing_interceptions', 'sacks_suffered', 'sack_yards_lost', 'sack_fumbles', 'sack_fumbles_lost', 'passing_air_yards', 'passing_yards_after_catch', 'passing_first_downs', 'passing_epa', 'passing_cpoe', 'passing_2pt_conversions', 'pacr', 'carries', 'rushing_yards', 'rushing_tds', 'rushing_fumbles', 'rushing_fumbles_lost', 'rushing_first_downs', 'rushing_epa', 'rushing_2pt_conversions', 'receptions', 'targets', 'receiving_yards', 'receiving_tds', 'receiving_fumbles', 'receiving_fumbles_lost', 'receiving_air_yards', 'receiving_yards_after_catch', 'receiving_first_downs', 'receiving_epa', 'receiving_2pt_conversions', 'racr', 'target_share', 'air_yards_share', 'wopr', 'special_teams_tds', 'def_tackles_solo', 'def_tackles_with_assist', 'def_tackle_assists'

player_name,position,team_clean,week
str,str,str,i32
"""Aaron Rodgers""","""QB""","""PIT""",1
"""Matt Prater""","""K""","""BUF""",1
"""Nick Folk""","""K""","""NYJ""",1
"""Joe Flacco""","""QB""","""CLE""",1
"""Calais Campbell""","""DE""","""ARI""",1
"""Matthew Stafford""","""QB""","""LA""",1
"""Graham Gano""","""K""","""NYG""",1
"""Thomas Morstead""","""P""","""SF""",1
"""Trent Williams""","""OT""","""SF""",1


In [16]:
# Cell — build per-game team totals using only columns that exist
import polars as pl

def col_exists(df: pl.DataFrame, name: str) -> bool:
    return name in df.columns

# We'll aggregate per (season, week, team_clean)
aggs = []

# team pass attempts
if col_exists(pw_hist, "attempts"):
    aggs.append(pl.sum("attempts").alias("team_pass_att"))
else:
    # create a constant 0 if not present, so downstream code doesn't break
    aggs.append(pl.lit(0).alias("team_pass_att"))

# team rush attempts
if col_exists(pw_hist, "rushing_attempts"):
    aggs.append(pl.sum("rushing_attempts").alias("team_rush_att"))
else:
    aggs.append(pl.lit(0).alias("team_rush_att"))

# team passing TDs and rushing TDs (we'll sum them into team_off_tds)
have_ptd = col_exists(pw_hist, "passing_tds")
have_rtd = col_exists(pw_hist, "rushing_tds")

agg_ptd = pl.sum("passing_tds").alias("team_pass_tds") if have_ptd else pl.lit(0).alias("team_pass_tds")
agg_rtd = pl.sum("rushing_tds").alias("team_rush_tds") if have_rtd else pl.lit(0).alias("team_rush_tds")
aggs.extend([agg_ptd, agg_rtd])

team_g = (
    pw_hist
    .group_by(["season","week","team_clean"])
    .agg(aggs)
    .with_columns([
        (pl.col("team_pass_tds") + pl.col("team_rush_tds")).alias("team_off_tds")
    ])
)

print("Built team_g with columns:", team_g.columns)
team_g.head(10)


Built team_g with columns: ['season', 'week', 'team_clean', 'team_pass_att', 'team_rush_att', 'team_pass_tds', 'team_rush_tds', 'team_off_tds']


season,week,team_clean,team_pass_att,team_rush_att,team_pass_tds,team_rush_tds,team_off_tds
i32,i32,str,i32,i32,i32,i32,i32
2025,2,"""TEN""",33,0,1,0,1
2025,1,"""LV""",34,0,1,1,2
2025,1,"""PHI""",23,0,0,3,3
2025,2,"""WAS""",42,0,2,0,2
2025,1,"""HOU""",27,0,0,0,0
2025,1,"""TEN""",28,0,0,0,0
2025,2,"""LA""",33,0,2,2,4
2025,1,"""DET""",39,0,1,0,1
2025,2,"""NO""",34,0,3,0,3


In [18]:
# Define exactly the players you want projections for.
# If a name doesn't match, run:
#   player_weekly.select("player_display_name").unique().sort("player_display_name")
# to find the exact display name used in the data.

MY_PLAYERS = [
    # WRs
    "Tyreek Hill",
    "Cooper Kupp",
    "Stefon Diggs",
    "Zay Flowers",
    "Nico Collins",
    "Rome Odunze",

    # RBs
    "Derrick Henry",
    "Tony Pollard",
    "Travis Etienne Jr.",
    "Kenneth Walker III",
    "James Conner",

    # TE
    "George Kittle",
    "Dallas Goedert",
    "David Njoku",

    # QB
    "Jalen Hurts",
]

# (Optional) quick sanity print
print("Players:", len(MY_PLAYERS))
for p in MY_PLAYERS: print("-", p)


Players: 15
- Tyreek Hill
- Cooper Kupp
- Stefon Diggs
- Zay Flowers
- Nico Collins
- Rome Odunze
- Derrick Henry
- Tony Pollard
- Travis Etienne Jr.
- Kenneth Walker III
- James Conner
- George Kittle
- Dallas Goedert
- David Njoku
- Jalen Hurts


In [19]:
# Cell — minimal per-player projection using only real columns and your Standard scoring

import polars as pl

# make sure MY_PLAYERS exists (list of strings) and LAST_DONE_WEEK/ROLL_N exist
try:
    MY_PLAYERS
except NameError:
    raise ValueError("Please define MY_PLAYERS = ['Tyreek Hill', ...] before running this cell.")

ROLL_N = 4 if 'ROLL_N' not in globals() else ROLL_N
LAST_DONE_WEEK = max(int(WEEK) - 1, 1) if 'LAST_DONE_WEEK' not in globals() else LAST_DONE_WEEK
start_wk = max(1, LAST_DONE_WEEK - ROLL_N + 1)

# helper to check column presence
def has(col: str) -> bool:
    return col in pw_hist.columns

# keep only last-N completed weeks for your players
hist = (
    pw_hist
    .filter((pl.col("player_name").is_in(MY_PLAYERS)) & (pl.col("week") >= start_wk) & (pl.col("week") <= LAST_DONE_WEEK))
    .select(["player_name","position","team_clean","week"] + [c for c in [
        "attempts","passing_yards","passing_tds","interceptions","two_point_conversions",
        "rushing_attempts","rushing_yards","rushing_tds","rushing_first_downs",
        "targets","receiving_yards","receiving_tds","receiving_first_downs"
    ] if has(c)])
)

# per-game averages over last-N window
avg = (
    hist.group_by(["player_name","position","team_clean"])
    .agg([
        pl.mean("attempts").alias("pa_g") if has("attempts") else pl.lit(0).alias("pa_g"),
        pl.mean("passing_yards").alias("py_g") if has("passing_yards") else pl.lit(0).alias("py_g"),
        pl.mean("passing_tds").alias("ptd_g") if has("passing_tds") else pl.lit(0).alias("ptd_g"),
        pl.mean("interceptions").alias("int_g") if has("interceptions") else pl.lit(0).alias("int_g"),
        pl.mean("two_point_conversions").alias("two_pt_g") if has("two_point_conversions") else pl.lit(0).alias("two_pt_g"),

        pl.mean("rushing_attempts").alias("ra_g") if has("rushing_attempts") else pl.lit(0).alias("ra_g"),
        pl.mean("rushing_yards").alias("ry_g") if has("rushing_yards") else pl.lit(0).alias("ry_g"),
        pl.mean("rushing_tds").alias("rtd_g") if has("rushing_tds") else pl.lit(0).alias("rtd_g"),
        pl.mean("rushing_first_downs").alias("rfd_g") if has("rushing_first_downs") else pl.lit(0).alias("rfd_g"),

        pl.mean("targets").alias("tgt_g") if has("targets") else pl.lit(0).alias("tgt_g"),
        pl.mean("receiving_yards").alias("rec_y_g") if has("receiving_yards") else pl.lit(0).alias("rec_y_g"),
        pl.mean("receiving_tds").alias("retd_g") if has("receiving_tds") else pl.lit(0).alias("retd_g"),
        pl.mean("receiving_first_downs").alias("recv_fd_g") if has("receiving_first_downs") else pl.lit(0).alias("recv_fd_g"),
    ])
)

# your league scoring (Standard / no PPR)
SCORING = {
    "pass_yd_per": 0.04,  # 1 pt per 25 pass yds
    "pass_td":     4.0,
    "int":        -2.0,
    "two_pt":      2.0,

    "rush_yd_per": 0.1,   # 1 pt per 10 rush yds
    "rush_td":     6.0,
    "rush_fd":     0.25,

    "rec_yd_per":  0.1,   # 1 pt per 10 rec yds
    "rec_td":      6.0,
    "rec_fd":      0.25
}

# compute projected fantasy points strictly from last-N per-game averages
proj = (
    avg.with_columns([
        # passing component
        (pl.col("py_g") * SCORING["pass_yd_per"] +
         pl.col("ptd_g") * SCORING["pass_td"] +
         pl.col("int_g") * SCORING["int"]).alias("fp_pass"),

        # rushing component
        (pl.col("ry_g") * SCORING["rush_yd_per"] +
         pl.col("rtd_g") * SCORING["rush_td"] +
         pl.col("rfd_g") * SCORING["rush_fd"]).alias("fp_rush"),

        # receiving component (no PPR)
        (pl.col("rec_y_g") * SCORING["rec_yd_per"] +
         pl.col("retd_g")  * SCORING["rec_td"] +
         pl.col("recv_fd_g") * SCORING["rec_fd"]).alias("fp_rec"),

        # 2-pt (from historical avg)
        (pl.col("two_pt_g") * SCORING["two_pt"]).alias("fp_two_pt"),
    ])
    .with_columns([
        (pl.col("fp_pass")+pl.col("fp_rush")+pl.col("fp_rec")+pl.col("fp_two_pt")).alias("proj_fp_lastN")
    ])
    .sort("proj_fp_lastN", descending=True)
)

# nice compact view
proj.select([
    "player_name","position","team_clean",
    "pa_g","py_g","ptd_g","int_g",
    "ra_g","ry_g","rtd_g","rfd_g",
    "tgt_g","rec_y_g","retd_g","recv_fd_g",
    "proj_fp_lastN"
])


player_name,position,team_clean,pa_g,py_g,ptd_g,int_g,ra_g,ry_g,rtd_g,rfd_g,tgt_g,rec_y_g,retd_g,recv_fd_g,proj_fp_lastN
str,str,str,f64,f64,f64,i32,i32,f64,f64,f64,f64,f64,f64,f64,f64
"""Jalen Hurts""","""QB""","""PHI""",22.5,126.5,0.0,0,0,38.5,1.5,6.0,0.0,0.0,0.0,0.0,19.41
"""Rome Odunze""","""WR""","""CHI""",0.0,0.0,0.0,0,0,0.0,0.0,0.0,10.0,82.5,1.5,4.5,18.375
"""Derrick Henry""","""RB""","""BAL""",0.0,0.0,0.0,0,0,96.0,1.0,3.0,0.5,6.5,0.0,0.5,17.125
"""Zay Flowers""","""WR""","""BAL""",0.0,0.0,0.0,0,0,6.0,0.0,0.0,10.0,109.0,0.5,4.0,15.5
"""James Conner""","""RB""","""ARI""",0.0,0.0,0.0,0,0,36.5,0.5,2.0,2.5,11.5,0.5,1.5,11.675
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Nico Collins""","""WR""","""HOU""",0.0,0.0,0.0,0,0,0.0,0.0,0.0,7.0,38.5,0.5,1.0,7.1
"""Cooper Kupp""","""WR""","""SEA""",0.0,0.0,0.0,0,0,0.0,0.0,0.0,6.0,52.5,0.0,2.5,5.875
"""Stefon Diggs""","""WR""","""NE""",0.0,0.0,0.0,0,0,0.0,0.0,0.0,6.0,44.5,0.0,2.0,4.95


In [20]:
# Cell — Print ACTUAL fantasy points for Weeks 1 & 2 (your Standard scoring), nicely formatted (not a DataFrame)

import polars as pl

# Scoring (from your screenshot): Standard / no PPR
SCORING = {
    "pass_yd_per": 0.04,  # 1 pt per 25 pass yds
    "pass_td":     4.0,
    "int":        -2.0,
    "two_pt":      2.0,

    "rush_yd_per": 0.1,   # 1 pt per 10 rush yds
    "rush_td":     6.0,
    "rush_fd":     0.25,

    "rec_yd_per":  0.1,   # 1 pt per 10 rec yds
    "rec_td":      6.0,
    "rec_fd":      0.25
}

# Ensure we have the player-week table for raw stats (player_weekly was loaded earlier)
# Build a minimal table with only the columns we need; create 0 columns if missing.
def ensure_cols(df: pl.DataFrame, cols: list[str]) -> pl.DataFrame:
    out = df
    for c in cols:
        if c not in out.columns:
            out = out.with_columns(pl.lit(0).alias(c))
    return out

need = [
    "player_display_name","position","team","recent_team","week",
    "passing_yards","passing_tds","interceptions","two_point_conversions",
    "rushing_yards","rushing_tds","rushing_first_downs",
    "receiving_yards","receiving_tds","receiving_first_downs"
]
pw_needed = ensure_cols(player_weekly, need)

# Use "team" if present, else recent_team
team_col = "team" if "team" in pw_needed.columns else "recent_team"

# Filter to your players & weeks 1-2
w12 = (
    pw_needed
    .filter( (pl.col("player_display_name").is_in(MY_PLAYERS)) & (pl.col("week").is_in([1,2])) )
    .with_columns([
        pl.col("player_display_name").alias("player_name"),
        pl.col(team_col).alias("team_clean")
    ])
    .select([
        "player_name","position","team_clean","week",
        "passing_yards","passing_tds","interceptions","two_point_conversions",
        "rushing_yards","rushing_tds","rushing_first_downs",
        "receiving_yards","receiving_tds","receiving_first_downs"
    ])
)

# Compute fantasy points for each player/week using your scoring
w12_fp = (
    w12.with_columns([
        (pl.col("passing_yards") * SCORING["pass_yd_per"]
         + pl.col("passing_tds") * SCORING["pass_td"]
         + pl.col("interceptions") * SCORING["int"]
         + pl.col("two_point_conversions") * SCORING["two_pt"]).alias("fp_pass"),

        (pl.col("rushing_yards") * SCORING["rush_yd_per"]
         + pl.col("rushing_tds") * SCORING["rush_td"]
         + pl.col("rushing_first_downs") * SCORING["rush_fd"]).alias("fp_rush"),

        (pl.col("receiving_yards") * SCORING["rec_yd_per"]
         + pl.col("receiving_tds") * SCORING["rec_td"]
         + pl.col("receiving_first_downs") * SCORING["rec_fd"]).alias("fp_rec"),
    ])
    .with_columns([
        (pl.col("fp_pass") + pl.col("fp_rush") + pl.col("fp_rec")).alias("fantasy_points")
    ])
    .sort(["week","player_name"])
)

# Pretty print (not a DataFrame)
def print_week_points(df: pl.DataFrame, week: int):
    rows = df.filter(pl.col("week")==week).to_dicts()
    print(f"\n=== Week {week} — Actual Fantasy Points (Standard) ===")
    for r in rows:
        pts = round(r["fantasy_points"], 2)
        print(f"• {r['player_name']} ({r['position']} — {r['team_clean']}): {pts}")

print_week_points(w12_fp, 1)
print_week_points(w12_fp, 2)



=== Week 1 — Actual Fantasy Points (Standard) ===
• Cooper Kupp (WR — SEA): 1.5
• Dallas Goedert (TE — PHI): 4.9
• David Njoku (TE — CLE): 3.95
• Derrick Henry (RB — BAL): 31.7
• George Kittle (TE — SF): 9.25
• Jalen Hurts (QB — PHI): 26.03
• James Conner (RB — ARI): 11.15
• Kenneth Walker III (RB — SEA): 2.4
• Nico Collins (WR — HOU): 2.5
• Rome Odunze (WR — CHI): 10.45
• Stefon Diggs (WR — NE): 6.45
• Tony Pollard (RB — TEN): 9.65
• Tyreek Hill (WR — MIA): 4.75
• Zay Flowers (WR — BAL): 22.35

=== Week 2 — Actual Fantasy Points (Standard) ===
• Cooper Kupp (WR — SEA): 10.25
• David Njoku (TE — CLE): 4.25
• Derrick Henry (RB — BAL): 2.55
• Jalen Hurts (QB — PHI): 12.79
• James Conner (RB — ARI): 12.2
• Kenneth Walker III (RB — SEA): 19.55
• Nico Collins (WR — HOU): 11.7
• Rome Odunze (WR — CHI): 26.3
• Stefon Diggs (WR — NE): 3.45
• Tony Pollard (RB — TEN): 10.45
• Tyreek Hill (WR — MIA): 11.9
• Zay Flowers (WR — BAL): 8.65
